# Beam Control 3D Render

Welcome to this interactive Jupyter notebook, designed to showcase the simulation and real-time 3D visualization of a particle beam control system. This notebook provides a comprehensive workflow for setting up and running the beam control system, visualizing it, and integrating 3D rendering with simulation data. This notebook is designed to help you visualize beam dynamics, adjust control parameters, and observe real-time system changes.

The visualization is built using WebGL and JavaScript to create an immersive and responsive experience. By the end of this notebook, you will have a fully operational beam control simulation environment, with the ability to visualize and interact with the beam dynamics in real-time.

## What to Expect from This Notebook

In this notebook, you will:

- **Explore the Setup Process:** Learn how to initialize the simulation environment by importing necessary Python modules, configuring logging, and setting up the project structure. The notebook ensures that the environment is ready for both simulation and visualization tasks.
- **Understand the Simulation Loop:** Dive into the core simulation logic, where the beam control environment is stepped through time, processing control actions, updating observations, and computing rewards. Detailed logging provides insights into the simulation's progress.
- **Interact with the Visualization System:** Experience real-time 3D visualization of the beamline through a web-based interface. The notebook integrates a WebSocket-based visualization wrapper, allowing dynamic updates of the beam trajectory and particle distribution as control actions are applied.
- **Run Simulations Asynchronously:** Learn how to run the simulation in the background without blocking the notebook, enabling you to interact with other cells while the simulation progresses.
- **Visualize Results in Real-Time:** Use an embedded iframe to access a web application that displays a 3D visualization of the beamline, complete with a control panel for adjusting magnet strengths and steering angles, and a scatter plot of particle positions.


The cell bellow imports necessary modules for the simulation, including asyncio for asynchronous execution, yaml for configuration management,
and logging for debugging. It also adds the project root to the Python path to ensure module imports work correctly.


In [1]:
import asyncio
import logging
import os
import sys
from pathlib import Path

import yaml
from IPython.display import HTML, display

from beam_3d_visualizer.beam_server.beam_visualization_wrapper import (
    BeamVisualizationWrapper,
)
from src.environments import ea

# Configure logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s"
)
logger = logging.getLogger(__name__)

## Setup Simulation Run-loop

Setupthe main simulation loop: initializing the environment, updating visuals, processing control inputs, and logging step-wise data until completion.

**Expected Output:**

- Console messages indicating the start of the simulation and step-by-step log information such as received actions, computed rewards, and updated observations.
- If a WebSocket client is connected, visualization updates will be sent accordingly.


In [2]:
async def run_simulation(env: ea.TransverseTuning):
    """Run the simulation loop, stepping the environment with control actions."""
    print("\n--- Starting simulation loop ---")

    step_count = 0
    done = False

    # Reset the environment
    observation, _ = env.reset()

    while not done:
        # Render and broadcast data to clients
        # Calls BeamVisualizationWrapper.render(),
        # which delegates to WebSocketWrapper.render()
        await env.render()

        # Check if we have a new control action from WebSocket
        if env.control_action is not None:
            print(f"New action received: {env.control_action}")
            env.last_action = env.control_action  # Update last_action with new action
            env.control_action = None  # Clear after use

        # Step through the environment with the last action
        # Note: last_action persists and is reused if no new action is received
        observation, reward, terminated, truncated, info = env.step(env.last_action)

        # Update step count
        step_count += 1

        logger.info(
            "Step %d: Action = %s, Reward = %s, Observation = %s",
            step_count,
            env.last_action,
            reward,
            observation,
        )

        done = truncated  # or terminated

    env.close()
    print("Simulation completed.")

The main function below sets up the beam control environment by loading the AREAS Transverse Tunning environment, which handles initializing the simulation environment. It ensures the WebSocket-based visualization wrapper is correctly set up before running the simulation.

**Expected Output:**

- A message indicating that the WebSocket server is waiting for a client connection.
- Logging messages showing WebSocket and visualization wrapper initialization.
- A confirmation message once the simulation starts.


In [3]:
async def main():
    """Main entry point to set up the environment and start the simulation."""
    # Initialize the environment and wrap it with BeamVisualizationWrapper
    env = ea.TransverseTuning(
        backend="cheetah",
        action_mode="direct",
        magnet_init_mode=None,
        render_mode="rgb_array",  # "human",
        backend_args={"generate_screen_images": True},
    )
    env = BeamVisualizationWrapper(env)

    if not env.env.connected:
        print("Waiting for WebSocket client to connect...")
        await asyncio.sleep(5.0)  # Small delay to prevent CPU overload

    # Run the simulation
    await run_simulation(env)

    print("Simulation shutdown completed.")

## Start Simulation Run

This cell asynchronously starts the simulation in the background and allows users to run other cells without blocking execution. A short delay is added to ensure WebSocket services are properly started.

**Expected Output:**

- Log messages confirming WebSocket server startup and connection establishment.

- If successful, the WebSocket client (JavaScript-based visualization) connects, and the event display (3D visualization of the beamline) updates accordingly.


In [ ]:
# Create a background task for the simulation
simulation_task = asyncio.create_task(main())

# Add a small delay to allow the web server and WebSocket to start
await asyncio.sleep(2.0)  # Adjust delay as needed

print("Simulation is running in the background. You can now run other cells.")

INFO:beam_3d_visualizer.beam_server.websocket_wrapper:WebSocket server thread started on ws://localhost:8081
INFO:beam_3d_visualizer.beam_server.beam_visualization_wrapper:Started JavaScript web application on http://localhost:5173
INFO:websockets.server:server listening on 127.0.0.1:8081
INFO:websockets.server:server listening on [::1]:8081
INFO:beam_3d_visualizer.beam_server.websocket_wrapper:WebSocket server running on ws://localhost:8081
ic| self.assets_dir: PosixPath('/Users/jankaiser/Documents/DESY/rl4aa25-tutorial/beam_3d_visualizer/beam_server/assets')
INFO:beam_3d_visualizer.beam_server.segment_3d_builder:Added Quadrupole: AREAMQZM1 at position 0.17504000663757324
INFO:beam_3d_visualizer.beam_server.segment_3d_builder:Added Quadrupole: AREAMQZM2 at position 0.7250400111079216
INFO:beam_3d_visualizer.beam_server.segment_3d_builder:Added VerticalCorrector: AREAMCVM1 at position 1.0510400086641312
INFO:beam_3d_visualizer.beam_server.segment_3d_builder:Added Quadrupole: AREAMQZM3 

Waiting for WebSocket client to connect...


INFO:websockets.server:connection open
INFO:beam_3d_visualizer.beam_server.websocket_wrapper:WebSocket connection established.


Simulation is running in the background. You can now run other cells.



--- Starting simulation loop ---


...

ERROR:asyncio:Fatal write error on socket transport
protocol: <websockets.asyncio.server.ServerConnection object at 0x1788f9e80>
transport: <_SelectorSocketTransport closing fd=86 read=idle write=<idle, bufsize=0>>
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/rl4aa25-tutorial/lib/python3.12/asyncio/selector_events.py", line 1103, in _write_sendmsg
    self._adjust_leftover_buffer(nbytes)
  File "/opt/homebrew/Caskroom/miniforge/base/envs/rl4aa25-tutorial/lib/python3.12/asyncio/selector_events.py", line 1128, in _adjust_leftover_buffer
    b = buffer.popleft()
        ^^^^^^^^^^^^^^^^
IndexError: pop from an empty deque
INFO:beam_3d_visualizer.beam_server.websocket_wrapper:WebSocket connection closed by client.
INFO:beam_3d_visualizer.beam_server.websocket_wrapper:Client cleanup completed.
INFO:websockets.server:connection open
INFO:beam_3d_visualizer.beam_server.websocket_wrapper:WebSocket connection established.
INFO:__main__:Step 65: Action = 

New action received: [0.         0.         0.         0.         0.00098851]


INFO:websockets.server:connection open
INFO:beam_3d_visualizer.beam_server.websocket_wrapper:WebSocket connection established.
INFO:__main__:Step 82: Action = [ 0.          0.          0.          0.         -0.00037069], Reward = 1.0, Observation = {'beam': array([ 0.00084966,  0.00013523, -0.00013569,  0.00012266], dtype=float32), 'magnets': array([ 0.        ,  0.        ,  0.        ,  0.        , -0.00037069],
      dtype=float32), 'target': array([-0.00181534,  0.00198796,  0.00064197,  0.00138882], dtype=float32)}


New action received: [ 0.          0.          0.          0.         -0.00037069]


INFO:beam_3d_visualizer.beam_server.websocket_wrapper:WebSocket connection closed by client.
INFO:beam_3d_visualizer.beam_server.websocket_wrapper:Client cleanup completed.
ERROR:asyncio:Fatal write error on socket transport
protocol: <websockets.asyncio.server.ServerConnection object at 0x1788fb1d0>
transport: <_SelectorSocketTransport closing fd=86 read=idle write=<idle, bufsize=0>>
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/rl4aa25-tutorial/lib/python3.12/asyncio/selector_events.py", line 1103, in _write_sendmsg
    self._adjust_leftover_buffer(nbytes)
  File "/opt/homebrew/Caskroom/miniforge/base/envs/rl4aa25-tutorial/lib/python3.12/asyncio/selector_events.py", line 1128, in _adjust_leftover_buffer
    b = buffer.popleft()
        ^^^^^^^^^^^^^^^^
IndexError: pop from an empty deque
INFO:beam_3d_visualizer.beam_server.websocket_wrapper:WebSocket connection closed by client.
INFO:beam_3d_visualizer.beam_server.websocket_wrapper:Client clean

## Beam Trajectory Visualization

Expected Visualization Output

- A real-time 3D visualization of the beamline appears in the web application.
- A control panel enables users to adjust magnet strengths and steering angles, updating the particle beam trajectory dynamically.
- A scatter plot of particle positions is displayed, showing the beam’s distribution in real-time.


<iframe src="http://localhost:5173" title="3D Lattice Visualization" height="600" width="1000"/>
